In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D,Dropout,Flatten,Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os


In [6]:

DIRECTORY = "dataset"
CATEGORIES = ["with_mask", "without_mask"]

data = []
labels = []
#iterating through each image and storing their data and
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(category)

# one hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# converting them to numpy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)


In [5]:
#Splitting the data to train and test
(trainX, testX, trainY, testY) = train_test_split(data, labels,
test_size=0.20, stratify=labels, random_state=42)

#intializing the epochs,batch size
epochs = 20
batch_size = 32

#Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

training_data = datagen.flow(trainX, trainY, batch_size=batch_size)
#loadin the Mobilenet model
mobilenet = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

#freezing the model
for layer in mobilenet.layers:
    layer.trainable = False

#creating the model
model = mobilenet.output
model = AveragePooling2D(pool_size=(7, 7))(model)
model = Flatten(name="flatten")(model)
model = Dense(128, activation="relu")(model)
model = Dropout(0.5)(model)
model = Dense(2, activation="sigmoid")(model)
final_model = Model(inputs=mobilenet.input, outputs=model)
opt = Adam(lr=1e-4, decay=1e-4 / epochs)

#compiling the model
final_model.compile(loss="binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

#training the model
history = final_model.fit(
    training_data,
    steps_per_epoch=len(trainX) // batch_size,
    validation_data=(testX, testY),
    validation_steps=len(testX) // batch_size,
    epochs=epochs)


In [ ]:

predicted = model.predict(testX, batch_size=BS)

predicted = np.argmax(predicted, axis=1)

print(classification_report(testY.argmax(axis=1), predicted,target_names=lb.classes_))




In [ ]:
# saving the model 
final_model.save("face_mask_detector.model", save_format="h5")
print("Model Saved")